In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
from time import time, sleep
from datetime import datetime
import random
import base64

In [2]:
# list of free proxies 
proxies = [{'http': '95.178.108.189'},
          {'http': '80.48.119.28'},
          {'http': '79.133.202.7'},
          {'http': '185.237.20.218'},
          {'http': '91.211.100.35'},
          {'http': '83.168.84.82'},
          {'http': '78.133.163.190'},
          {'http': '195.205.33.237'},
          {'http': '80.54.62.254'},
          {'http': '185.89.65.170'},
          {'http': '178.212.48.70'},
          {'http': '185.14.149.178'},
          {'http': '185.21.39.42'},
          {'http': '195.136.186.17'},
          {'http': '176.58.17.1'},
          {'http': '178.212.48.23'},
          {'http': '185.56.246.76'},
          {'http': '185.179.132.152'},
          {'http': '91.207.68.197'},
          {'http': '91.246.213.104'},
          {'http': '176.58.20.226'},
          {'http': '178.212.48.17'},
          {'http': '91.150.189.122'},
          {'http': '85.128.16.50'},
          {'http': '156.67.107.79'},
          {'http': '91.227.103.134'},
          {'http': '89.186.1.172'},
          {'http': '178.212.54.137'},
          {'http': '37.98.218.137'},
          {'http': '85.128.16.163'},
          {'http': '89.186.17.82'},
          {'http': '185.122.144.144'},
          {'http': '91.238.55.86'},
          {'http': '176.103.74.89'},
          {'http': '193.59.46.199'},
          {'http': '94.230.27.58'},
          {'http': '91.202.230.219'},
          {'http': '46.227.37.149'},
          {'http': '37.131.164.102'},
          {'http': '185.89.65.165'},
          {'http': '83.12.174.202'},
          {'http': '80.53.255.194'},
          {'http': '193.106.130.35'},
          {'http': '46.171.28.162'},
          {'http': '83.168.84.89'},
          {'http': '91.224.197.180'},
          {'http': '77.252.26.70'},
          {'http': '193.59.27.203'},
          {'http': '46.238.230.4'},
          {'http': '94.240.10.115'},
          {'http': '93.113.101.85'},
          {'http': '94.240.29.203'},
          {'http': '212.180.252.34'},
          {'http': '194.31.33.50'},
          {'http': '91.205.72.103'},
          {'http': '83.238.13.109'},
          {'http': '89.171.144.168'},
          {'http': '80.48.255.145'},
          {'http': '83.168.84.137'},
          {'http': '5.226.125.10'},
          {'http': '83.168.84.141'},
          {'http': '83.168.84.142'},
          {'http': '185.46.170.253'},
          {'http': '91.216.254.5'},
          {'http': '89.228.128.216'},
          {'http': '46.174.235.37'},
          {'http': '46.174.235.235'},
          {'http': '185.238.239.55'},
          {'http': '185.32.6.129'},
          {'http': '195.177.217.131'},
          {'http': '185.89.64.17'},
          {'http': '78.133.186.5'},
          {'http': '176.58.20.54'},
          {'http': '79.133.200.147'},
          {'http': '193.239.101.7'},
          {'http': '193.59.26.138'},
          {'http': '195.242.138.20'},
          {'http': '178.216.24.80'},
          {'http': '78.31.159.227'},
          {'http': '93.105.40.62'},
          {'http': '178.19.179.30'},
          {'http': '91.226.51.200'},
          {'http': '5.172.188.90'},
          {'http': '62.133.135.129'},
          {'http': '93.105.171.100'},
          {'http': '178.212.52.67'},
          {'http': '46.174.234.32'},
          {'http': '91.234.127.222'},
          {'http': '5.226.92.74'},
          {'http': '147.135.209.179'},
          {'http': '94.23.91.209'}]

In [3]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

In [26]:
def scrap_eobuwie(brands):

    start = time()
    
    session = requests.Session()
    session.max_redirects = 90
    cookies = dict(cookies_are='working')
        
    # empty dataframe to collect data
    pricelist = pd.DataFrame({"Article" : [],
                 "Price" : [],
                 "RRP": [],
                 "Long_name": [],
                 "Brand": []})

    # iterate through each provided brand
    for id, brand in enumerate(brands):

      # iterate through each subpage
      for n in range(1, 100):
        url = "https://www.eobuwie.com.pl/" + brand + ".html?p=" + str(n)

        # get whole structure of data, and avoid getting blocked, by add headers parameter
        page = session.get(url, headers = headers,cookies=cookies) 

        # let parse our web page
        soup = BeautifulSoup(page.content)

        # iterate through all products on one page to get direct link to them
        for product in soup.find_all(attrs =  {"class": "products-list__link"}):
          try:
            link = 'http://' + product['href'][2:]

            try:
              page_article = session.get(link, headers = headers, timeout = 98,cookies=cookies) 
              soup_article = BeautifulSoup(page_article.content)

            except:
              # choose random proxy
              proxy = random.choice(proxies) 

              session = requests.Session()
              session.max_redirects = 90
              cookies = dict(cookies_are='working') 

              page_article = session.get(link, headers = headers, timeout = 98,cookies=cookies, proxies = proxy) 
              soup_article = BeautifulSoup(page_article.content)

            # getting information about articels
            try:
              try:
                RRP = soup_article.find(attrs = {"class" : "e-product-price__group"}).find(attrs = {"class" : "e-product-price__regular"}).text.strip().replace('\xa0zł',"")
              except:
                RRP = soup_article.find(attrs = {"class" : "e-product-price__normal"}).text.strip().replace('\xa0zł',"")

              try:
                PRICE = soup_article.find(attrs = {"class" : "e-product-price__group"}).find(attrs = {"class" : "e-product-price__special"}).text.strip().replace('\xa0zł',"")
              except:
                PRICE = RRP

              NAME = soup_article.find("meta" ,property="product:mfr_part_no")["content"]

              LONG_NAME = soup_article.find(attrs = {"class" : "e-product-name__model"}).text.strip()

              pricelist = pricelist.append({'Article' : NAME,
                                    'Price' : PRICE,
                                    'RRP' : RRP,
                                    'Long_name': LONG_NAME,
                                    'Brand': brand},
                                    ignore_index = True)
                
            except:
              continue #if article is sold out
            
            # sleep for one seconds after each link to avoid block
            sleep(3)

          except: #if url doesn't work
            continue

        # check if there is another subpage
        try:
          next_page = soup.find(class_='toolbar-bottom__pager').find_all('a')[-1].text.strip()

          if next_page != '»' or next_page == str(n):
            break
        except:
          break

        print("number of subpage: ", brand, n) 
        sleep(5)

    end = time()
    print(round(end - start,2), 'calculation time in sec')
    
    return pricelist

In [ ]:
brands = ["adidas_performance","puma", "reebok","4f","kappa","new_balance","skechers","under_armour","vans","lee_cooper","ipanema","campus"]

In [ ]:
puma = scrap_eobuwie(["puma"])

number of subpage:  puma 1
number of subpage:  puma 2
number of subpage:  puma 3
number of subpage:  puma 4
number of subpage:  puma 5
number of subpage:  puma 6
number of subpage:  puma 7
number of subpage:  puma 8
4431.78 calculation time in sec


In [ ]:
adidas = scrap_eobuwie(["adidas"])

number of subpage:  adidas 1
number of subpage:  adidas 2
number of subpage:  adidas 3
number of subpage:  adidas 4
number of subpage:  adidas 5
number of subpage:  adidas 6
number of subpage:  adidas 7
number of subpage:  adidas 8
number of subpage:  adidas 9
number of subpage:  adidas 10
number of subpage:  adidas 11
number of subpage:  adidas 12
number of subpage:  adidas 13
number of subpage:  adidas 14
number of subpage:  adidas 15
number of subpage:  adidas 16
number of subpage:  adidas 17
number of subpage:  adidas 18
9815.89 calculation time in sec


In [6]:
reebok = scrap_eobuwie(["reebok"])

number of subpage:  reebok 1
number of subpage:  reebok 2
number of subpage:  reebok 3
number of subpage:  reebok 4
number of subpage:  reebok 5
2763.42 calculation time in sec


In [7]:
cztery_f = scrap_eobuwie(["4f"])

number of subpage:  4f 1
number of subpage:  4f 2
1157.41 calculation time in sec


In [8]:
kappa = scrap_eobuwie(["kappa"])

number of subpage:  kappa 1
895.77 calculation time in sec


In [10]:
skechers = scrap_eobuwie(["skechers"])

number of subpage:  skechers 1
number of subpage:  skechers 2
number of subpage:  skechers 3
number of subpage:  skechers 4
number of subpage:  skechers 5
number of subpage:  skechers 6
number of subpage:  skechers 7
number of subpage:  skechers 8
number of subpage:  skechers 9
number of subpage:  skechers 10
5236.97 calculation time in sec


In [9]:
new_balance= scrap_eobuwie(["new_balance"])

number of subpage:  new_balance 1
number of subpage:  new_balance 2
number of subpage:  new_balance 3
number of subpage:  new_balance 4
number of subpage:  new_balance 5
number of subpage:  new_balance 6
number of subpage:  new_balance 7
number of subpage:  new_balance 8
number of subpage:  new_balance 9


AttributeError: ignored

In [11]:
campus= scrap_eobuwie(["campus"])

AttributeError: ignored

In [20]:
link = "https://www.eobuwie.com.pl/skechers.html?p=11"

In [22]:
page = requests.get(link, headers = headers, timeout = 98) 
soup = BeautifulSoup(page.content)
next_page = soup.find(class_='toolbar-bottom__pager').find_all('a')[-1].text.strip()
next_page

'10'

In [25]:
try:
  next_page = soup.find(class_='toolbar-bottom__pager').find_all('a')[-1].text.strip()

  if next_page != '»' or next_page == str(10):
    print("xxx")
except:
  print("Ddd")

xxx


In [12]:
ipanema= scrap_eobuwie(["ipanema"])

number of subpage:  ipanema 1
646.89 calculation time in sec


In [ ]:
data = reebok.append(cztery_f, kappa,skechers, new_balance,ipanema)

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
data.to_csv("Eobuwie_" + today + ".csv",index = False)